In [45]:
import numpy as np
import pandas as pd



In [46]:
# initialize fees
FEE_F = 0.003

# initialize basic arrays and dictionaries
q = {'ab': [0., 0.], 'ac': [0., 0.], 'bc': [0.,0.]}
swap_amt = [0.]*2

# Test swap method, or not
TEST_SWAP_CODE = False

In [47]:
# Experiment1 (and initialize dataframe)

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

init_state = {
    'exch': ['ab', 'ac', 'bc'],
    'tok0': [0.]*3,
    'tok1': [0.]*3,
    'res0': [q['ab'][0], q['ac'][0], q['bc'][0]],
    'res1': [q['ab'][1], q['ac'][1], q['bc'][1]],
    'reason': ["exp1"]*3
}

ledger_df = pd.DataFrame(init_state)

print(ledger_df)


  exch  tok0  tok1   res0   res1 reason
0   ab   0.0   0.0  100.0  100.0   exp1
1   ac   0.0   0.0  200.0  200.0   exp1
2   bc   0.0   0.0  250.0  250.0   exp1


In [48]:
# Swap code
#     Follows the Uniswap code

def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0
    assert x < 1
    return 1/(1-x)

def swap(data_):
    # data_ = { 'reserves': q['ab'], 'legs': [10., 0.], 'direction': 'forward', 'fee': FEE_F}
    f_ = data_['fee']
    d_ = data_['legs']
    r_ = data_['reserves']
    direction:str = data_['direction']

    assert r_[0] > 0
    assert r_[1] > 0

    # The in_leg has the non-zero entry
    if d_[0] > 0:
        assert d_[1] == 0
        in_leg = 0
        out_leg = 1
    else:
        assert d_[0] == 0
        in_leg = 1
        out_leg = 0

    q_in = r_[in_leg]
    q_out = r_[out_leg]

    if direction == 'backward':
        # backward swap:
        #     find the input that produces the output
        #     the output is known, so this is the "in_leg" which has a non-zero entry
        d_out = d_[in_leg]
        result_idx = out_leg
        d_in = float((1 / (1 - f_)) * (q_in / q_out) * phi(d_out / q_out) * d_out)
        d_[out_leg] = d_in
    elif direction == 'forward':
        # forward swap:
        #     find the output produced by the input
        #     the input is known, so this is the "in_leg" which has a non-zero entry
        d_in = d_[in_leg]
        result_idx = out_leg
        d_in = (1 - f_) * d_in
        d_out = float((q_out / q_in) * psi(d_in / q_in) * d_in)
        d_[out_leg] = d_out
    else:
        print('unknown direction')
        return

    r_[out_leg] -= d_out
    r_[in_leg]  += d_in
    return {'reserves': r_, 'legs': d_, 'result_idx': result_idx }


# Tests
if TEST_SWAP_CODE:
    q = reset_reserves(q)

    # swap a->b
    reason = 'route1: a->b'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
    print(reason)
    print(swap_out)
    route1_b_tokens = swap_out['legs'][swap_out['result_idx']]

    # swap c->b
    reason = 'cycle: c->b '
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    cycle_c1_tokens = swap_out['legs'][swap_out['result_idx']]
    print(reason)
    print(swap_out)
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")


In [49]:
def run_cycle_test(q):
    # Run two routes: a->b and a->c->b
    # Notice that a->c->b returns *more* b-tokens
    # Run cycle: c1->b->a->c2
    # Notice that c2 > c1

    # a->b route1
    q = reset_reserves(q)

    reason = 'route1: a->b'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [10., 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    route1_b_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"route1_b_tokens: {route1_b_tokens}")

    # a->c->b route2
    q = reset_reserves(q)

    # swap a->c
    reason = 'route2: a->c'
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [10., 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]

    # record route2 c-token result
    route2_c_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"route2_c_tokens: {route2_c_tokens}")

    # swap c->b
    reason = 'route2: c->b'
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route2_c_tokens], # note: incoming leg is c-tokens, which is index 1
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]

    route2_b_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"route2_b_tokens: {route2_b_tokens}")

    print(f"Difference b-tokens route1 minus route2: {route1_b_tokens - route2_b_tokens}")


    # cycle

    if route2_b_tokens - route1_b_tokens > 0:
        cycle = 'c->b->a->c'
    else:
        cycle = 'c->a->b->c'

    print(f"cycle: {cycle}")

    q = reset_reserves(q)

    if cycle == 'c->b->a->c':
        # swap c->b
        reason = 'cycle: c->b'
        exch = 'bc'
        swap_out = swap({ 'reserves': q[exch],
                      'legs': [route1_b_tokens, 0.],
                      'direction': 'backward',
                      'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
        cycle_c1_tokens = swap_out['legs'][swap_out['result_idx']]
        print(f"cycle_c1_tokens: {cycle_c1_tokens}")

        # swap b->a
        reason = 'cycle: b->a'
        exch = 'ab'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [0., route1_b_tokens],
                          'direction': 'backward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
        cycle_a_tokens = swap_out['legs'][swap_out['result_idx']]
        print(f"cycle_a_tokens: {cycle_a_tokens}")

        # swap a->c
        reason = 'cycle: a->c'
        exch = 'ac'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [cycle_a_tokens, 0.],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
        cycle_c2_tokens = swap_out['legs'][swap_out['result_idx']]
        print(f"cycle_c2_tokens: {cycle_c2_tokens}")

        print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")

        if cycle_c2_tokens - cycle_c1_tokens > 0:
            print("SUCCESS: cycle arbitrage from mispricing b-tokens")

In [50]:
# Experiment 1
#   Test if cheaper route can generate arbitrage cycle
#   Result: yes, but effect is minor
#   Difference c2 minus c1: 0.06103022180536222

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

run_cycle_test(q)

route1_b_tokens: 9.066108938801491
route2_c_tokens: 9.496594751631186
route2_b_tokens: 9.122609663880215
Difference b-tokens route1 minus route2: -0.056500725078723946
cycle: c->b->a->c
cycle_c1_tokens: 9.435564529825822
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.496594751631184
Difference c2 minus c1: 0.06103022180536222
SUCCESS: cycle arbitrage from mispricing b-tokens


In [51]:
# Experiment 2
#   Test if more liquidity in the c-token reserves makes a difference
#   It does:
#   Difference c2 minus c1: 0.7940602859152346

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [2000., 2000.]
    q_['bc'] = [2500., 2500.]
    return q_

q = reset_reserves(q)

for exch in ['ab', 'ac', 'bc']:
    state = [exch, 0., 0., q[exch][0], q[exch][1], 'exp2']
    ledger_df.loc[len(ledger_df)] = state

run_cycle_test(q)

route1_b_tokens: 9.066108938801491
route2_c_tokens: 9.920546077802156
route2_b_tokens: 9.85180759745414
Difference b-tokens route1 minus route2: -0.7856986586526489
cycle: c->b->a->c
cycle_c1_tokens: 9.12648579188692
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.920546077802154
Difference c2 minus c1: 0.7940602859152346
SUCCESS: cycle arbitrage from mispricing b-tokens


In [52]:
# Experiment 3
#   same as in Experiment 2 but hugely scaled
#   A bit more but not much:
#   Difference c2 minus c1: 0.8757841355703171
def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200000., 200000.]
    q_['bc'] = [250000., 250000.]
    return q_

q = reset_reserves(q)

for exch in ['ab', 'ac', 'bc']:
    state = [exch, 0., 0., q[exch][0], q[exch][1], 'exp3']
    ledger_df.loc[len(ledger_df)] = state

run_cycle_test(q)

route1_b_tokens: 9.066108938801491
route2_c_tokens: 9.969503020274441
route2_b_tokens: 9.939199344768605
Difference b-tokens route1 minus route2: -0.8730904059671136
cycle: c->b->a->c
cycle_c1_tokens: 9.093718884704122
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.96950302027444
Difference c2 minus c1: 0.8757841355703171
SUCCESS: cycle arbitrage from mispricing b-tokens


In [36]:
print(ledger_df)

   exch       tok0      tok1           res0           res1        reason
0    ab   0.000000  0.000000     100.000000     100.000000          exp1
1    ac   0.000000  0.000000     200.000000     200.000000          exp1
2    bc   0.000000  0.000000     250.000000     250.000000          exp1
3    ab  10.000000  9.066109     109.970000      90.933891  route1: a->b
4    ac  10.000000  9.496595     209.970000     190.503405  route2: a->c
5    bc   9.122610  9.496595     240.877390     259.468105  route2: c->b
6    ab  10.000000  9.066109     109.970000      90.933891  route1: a->b
7    ac  10.000000  9.496595     209.970000     190.503405  route2: a->c
8    bc   9.122610  9.496595     240.877390     259.468105  route2: c->b
9    bc   9.066109  9.435565     259.435565     240.933891   cycle: c->b
10   ab  10.000000  9.066109      90.933891     110.000000   cycle: b->a
11   ac  10.000000  9.496595     209.970000     190.503405   cycle: a->c
12   ab  10.000000  9.066109     109.970000      90

In [53]:
def run_post_trade_arb_test(q):

    # a->b route1
    q = reset_reserves(q)

    reason = 'route1: b->a'
    exch = 'ab'

    route1_b_tokens = 10.
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0, route1_b_tokens],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]

    route1_a_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"route1_a_tokens: {route1_a_tokens}")
    print(f"route1_b_tokens: {route1_b_tokens}")

    # run cycle to immunize a->b loss by c-token profit

    # swap c->b
    reason = 'cycle: c->b'
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_c1_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")

    # swap b->a
    reason = 'cycle: b->a'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route1_b_tokens],
                      'direction': 'backward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_a_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_a_tokens: {cycle_a_tokens}")

    # swap a->c
    reason = 'cycle: a->c'
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [cycle_a_tokens, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves'] + [reason]
    cycle_c2_tokens = swap_out['legs'][swap_out['result_idx']]
    print(f"cycle_c2_tokens: {cycle_c2_tokens}")

    print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")

    if cycle_c2_tokens - cycle_c1_tokens > 0:
        print("SUCCESS: post-trade c->b->a->c arbitrage following b->a")

In [54]:
# Experiment 4
#   Test if running a cycle arbitrage after a trade makes a profit
#   It does
#   Difference c2 minus c1: 3.42557216517843

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [2000., 2000.]
    q_['bc'] = [2500., 2500.]
    return q_

q = reset_reserves(q)

for exch in ['ab', 'ac', 'bc']:
    state = [exch, 0., 0., q[exch][0], q[exch][1], 'exp3']
    ledger_df.loc[len(ledger_df)] = state

run_post_trade_arb_test(q)

route1_a_tokens: 9.066108938801491
route1_b_tokens: 10.0
cycle_c1_tokens: 10.070371757843812
cycle_a_tokens: 13.628518444111364
cycle_c2_tokens: 13.495943923022242
Difference c2 minus c1: 3.42557216517843
SUCCESS: cycle arbitrage from mispricing b-tokens
